In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text

In [3]:
webData = BeautifulSoup(source, 'html.parser');

In [4]:
rows=[];
for tr in webData.table.tbody.find_all('tr'):
    cells=[]
    for td in tr.find_all('td'):        
        cells.append(td.text.replace('\n',''))
    rows.append(cells)

In [12]:
# Creating DataFrame
columns_name =['PostalCode','Borough','Neighborhood']
df = pd.DataFrame(data = rows, columns = columns_name)

# Drop first "none" row
df.drop([0],inplace=True)

# Drop rows from df where Borough == 'Not assigned'
df = df[df['Borough'] != 'Not assigned']

# If columns 'Neighborhood' equal 'Not assigned' should be changed by value from column 'Borough'
df['Neighborhood'][df['Neighborhood']=='Not assigned'] = df['Borough']

# Group the same values in "PostalCode", "Borough" fields
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame().reset_index()

df.head(12)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
print ("df.shape: ",df.shape)

df.shape:  (103, 3)


In [7]:
!wget -q -O 'latLong.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [8]:
df_latlong = pd.read_csv("latLong.csv")
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_full = df.set_index('PostalCode').join(df_latlong.set_index('Postal Code')).reset_index()
df_full.head(12)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
df_full.shape

(103, 5)